In [6]:
class MulLayer:
    def __init__(self):
        self.x = None
        self.y = None
        
    def forward(self, x, y):
        self.x = x
        self.y = y
        out = x * y
        
        return out
    
    def backward(self, dout):
        dx = dout * self.y
        dy = dout * self.x
        
        return dx, dy

In [7]:
apple = 100
apple_num = 2
tax = 1.1

In [8]:
mul_apple_layer = MulLayer()
mul_tax_layer = MulLayer()

In [9]:
apple_price = mul_apple_layer.forward(apple, apple_num)
price = mul_tax_layer.forward(apple_price, tax)

In [10]:
print(price)

220.00000000000003


In [11]:
dprice = 1

In [14]:
dapple_price, dtax = mul_tax_layer.backward(dprice)
dapple, dapple_num = mul_apple_layer.backward(dapple_price)

In [16]:
print(dapple, dapple_num, dtax)

2.2 110.00000000000001 200


In [17]:
class AddLayer:
    def __init__(self):
        pass
    
    def forward(self, x, y):
        out = x + y
        return out
    
    def backward(self, dout):
        dx = dout * 1
        dy = dout * 1
        return dx, dy

In [18]:
apple = 100
apple_num = 2
orange = 150
orange_num = 3
tax = 1.1

In [19]:
mul_apple_layer = MulLayer()
mul_orange_layer = MulLayer()
add_apple_orange_layer = AddLayer()
mul_tax_layer = MulLayer()

In [20]:
apple_price = mul_apple_layer.forward(apple, apple_num)
orange_price = mul_orange_layer.forward(orange, orange_num)
all_price = add_apple_orange_layer.forward(apple_price, orange_price)
price = mul_tax_layer.forward(all_price, tax)

In [21]:
dprice = 1
dall_price, dtax = mul_tax_layer.backward(dprice)
dapple_price, dorange_price = add_apple_orange_layer.backward(dall_price)
dorange, dorange_num = mul_orange_layer.backward(dorange_price)
dapple, dapple_num = mul_apple_layer.backward(dapple_price)

In [22]:
print(price)

715.0000000000001


In [23]:
print(dapple_num, dapple, dorange, dorange_num, dtax)

110.00000000000001 2.2 3.3000000000000003 165.0 650


In [24]:
class Relu:
    def __init__(self):
        self.mask = None
        
    def forward(self, x):
        self.mask = (x <= 0)
        out = x.copy()
        out[self.mask] = 0
        
        return out
    
    def backward(self, dout):
        dout[self.mask] = 0
        dx = dout
        
        return dx

In [26]:
import numpy as np
x = np.array([[1.0, -0.5], [-2.0, 3.0]])
print(x)

[[ 1.  -0.5]
 [-2.   3. ]]


In [27]:
mask = (x <= 0)
print(mask)

[[False  True]
 [ True False]]


In [28]:
class Sigmoid:
    def __init__(self):
        self.out = None
        
    def forward(self, x):
        out = 1 / (1 + np.exp(-x))
        self.out = out
        
        return out
    
    def backward(self, dout):
        dx = dout * (1.0 - self.out) * self.out
        
        return dx

In [29]:
X = np.random.rand(2)
W = np.random.rand(2, 3)
B = np.random.rand(3)

In [30]:
X.shape

(2,)

In [31]:
W.shape

(2, 3)

In [32]:
B.shape

(3,)

In [33]:
Y = np.dot(X, W) + B

In [34]:
Y

array([0.2146482 , 0.4974739 , 0.78495078])

In [35]:
X_dot_W = np.array([[0, 0, 0], [10, 10, 10]])

In [36]:
B = np.array([1, 2, 3])

In [37]:
X_dot_W

array([[ 0,  0,  0],
       [10, 10, 10]])

In [38]:
X_dot_W + B

array([[ 1,  2,  3],
       [11, 12, 13]])

In [39]:
dY = np.array([[1, 2, 3], [4, 5, 6]])

In [40]:
dY

array([[1, 2, 3],
       [4, 5, 6]])

In [41]:
dB = np.sum(dY, axis=0)

In [42]:
dB

array([5, 7, 9])

In [43]:
class Affine:
    def __init__(self, W, b):
        self.W = W
        self.b = b
        self.x = None
        self.dW = None
        self.db = None
        
    def forward(self, x):
        self.x = x
        out = np.dot(x, self.W) + self.b
        
        return out
    
    def backward(self, dout):
        dx = np.dot(dout, self.W.T)
        self.dW = np.dot(self.x.T, dout)
        self.db = np.sum(dout, axis=0)
        
        return dx

In [44]:
class SoftmaxWithLoss:
    def __init__(self):
        self.loss = None
        self.y = None
        self.t = None
        
    def forward(self, x, t):
        self.t = t
        self.y = softmax(x)
        self.loss = cross_entropy_error(self.y, self.t)
        
        return self.loss
    
    def backward(self, dout=1):
        batch_size = self.t.shape[0]
        dx = (self.y - self.t) / batch_size
        
        return dx